## 1. Import Dependencies

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [ ]:
%%script C:\Users\DELL\AppData\Local\Programs\Python\Python310\python.exe


In [7]:
pip install mediapipe

                                              0.0/50.2 MB ? eta -:--:--
                                              0.6/50.2 MB 18.5 MB/s eta 0:00:03
                                              0.8/50.2 MB 9.6 MB/s eta 0:00:06
                                              0.9/50.2 MB 7.5 MB/s eta 0:00:07
                                              1.1/50.2 MB 6.4 MB/s eta 0:00:08
     -                                        1.3/50.2 MB 5.9 MB/s eta 0:00:09
     -                                        1.5/50.2 MB 5.6 MB/s eta 0:00:09
     -                                        1.7/50.2 MB 5.3 MB/s eta 0:00:10
     -                                        1.9/50.2 MB 5.1 MB/s eta 0:00:10
     -                                        2.0/50.2 MB 5.0 MB/s eta 0:00:10
     -                                        2.2/50.2 MB 4.9 MB/s eta 0:00:10
     -                                        2.4/50.2 MB 4.8 MB/s eta 0:00:11
     --                                       2.6/50.2 MB 

In [8]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install --upgrade pip

  Using cached pip-23.1.2-py3-none-any.whl (2.1 MB)
  Attempting uninstall: pip
    Found existing installation: pip 22.3.1
    Uninstalling pip-22.3.1:
      Successfully uninstalled pip-22.3.1
Note: you may need to restart the kernel to use updated packages.


## 2. Keypoints using MP Holistic

In [10]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils #Drawing utilities

In [11]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [14]:

def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

In [16]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        #print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
             break
    cap.release()
    cv2.destroyAllWindows()

In [ ]:
len(results.pose_landmarks.landmark)

## 3. Extract Keypoint Values

In [17]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [18]:

result_test = extract_keypoints(results)

In [19]:
np.shape(result_test)

(1662,)

## 4. Setup Folders for Collection

In [20]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['hello', 'thanks', 'iloveyou','good','morning','afternoon','nice','meet','you','bye','how'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

In [21]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

## 5. Collect Keypoint Values for Training and Testing

In [22]:

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # Loop through actions
    for action in actions:
        # Loop through videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(500)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                #  Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()                  

In [23]:
cap.release()
cv2.destroyAllWindows()

## 6. Preprocess Data and Create Labels and Features

In [29]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [25]:
pip install scikit-learn

                                              0.0/8.3 MB ? eta -:--:--
     --                                       0.6/8.3 MB 18.5 MB/s eta 0:00:01
     ---                                      0.8/8.3 MB 9.6 MB/s eta 0:00:01
     ----                                     0.9/8.3 MB 7.8 MB/s eta 0:00:01
     ----                                     0.9/8.3 MB 7.8 MB/s eta 0:00:01
     ----                                     0.9/8.3 MB 4.2 MB/s eta 0:00:02
     ------                                   1.3/8.3 MB 5.0 MB/s eta 0:00:02
     -------                                  1.5/8.3 MB 4.8 MB/s eta 0:00:02
     --------                                 1.7/8.3 MB 4.7 MB/s eta 0:00:02
     ---------                                1.9/8.3 MB 4.6 MB/s eta 0:00:02
     ---------                                1.9/8.3 MB 4.6 MB/s eta 0:00:02
     ---------                                2.0/8.3 MB 3.9 MB/s eta 0:00:02
     ----------                               2.2/8.3 MB 4.1 MB/s eta 

In [28]:
pip install tensorflow==2.8

                                              0.0/438.0 MB ? eta -:--:--
                                             0.6/438.0 MB 18.2 MB/s eta 0:00:25
                                              0.7/438.0 MB 9.5 MB/s eta 0:00:47
                                              0.9/438.0 MB 7.3 MB/s eta 0:01:00
                                              1.1/438.0 MB 6.4 MB/s eta 0:01:08
                                              1.3/438.0 MB 5.4 MB/s eta 0:01:21
                                              1.5/438.0 MB 5.4 MB/s eta 0:01:21
                                              1.6/438.0 MB 5.2 MB/s eta 0:01:24
                                              1.8/438.0 MB 5.0 MB/s eta 0:01:27
                                              2.0/438.0 MB 4.9 MB/s eta 0:01:30
                                              2.2/438.0 MB 4.8 MB/s eta 0:01:31
                                              2.4/438.0 MB 4.7 MB/s eta 0:01:33
                                              2

Note: you may need to restart the kernel to use updated packages.


In [30]:

label_map = {label:num for num, label in enumerate(actions)}

In [31]:

label_map

{'hello': 0,
 'thanks': 1,
 'iloveyou': 2,
 'good': 3,
 'morning': 4,
 'afternoon': 5,
 'nice': 6,
 'meet': 7,
 'you': 8,
 'bye': 9,
 'how': 10}

In [33]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [34]:
np.array(sequences).shape #11actions *30 videos, 30 frames, 1662 keypointa

(330, 30, 1662)

In [35]:
np.array(labels).shape

(330,)

In [36]:
X = np.array(sequences)

In [37]:
X.shape

(330, 30, 1662)

In [38]:
y = to_categorical(labels).astype(int)

In [ ]:
y


In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [40]:
y_test.shape

(66, 11)

## 7. Build and train LSTM Neutral Network

In [41]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [42]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [43]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [44]:
model.fit(X_train, y_train, epochs=2)

Epoch 1/2
9/9 [==============================] - 5s 122ms/step - loss: 7.4070 - categorical_accuracy: 0.1023
Epoch 2/2
9/9 [==============================] - 1s 120ms/step - loss: 77.2255 - categorical_accuracy: 0.0871


In [45]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 30, 64)            442112    
                                                                 
 lstm_1 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 11)                363       
                                                                 
Total params: 596,939
Trainable params: 596,939
Non-trai


## 8. Make Predictions

In [46]:
res = model.predict(X_test)

In [47]:
model.save('action.h5')

In [48]:
model.load_weights('action.h5')

In [49]:
model.load_weights('action-Copy.h5')

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = 'action-Copy.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model('action.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("converted_model.tflite", "wb").write(tflite_model)

## 9. Evaluation using Confusion Matrix and Accuracy

In [50]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [51]:
yhat = model.predict(X_test)

In [52]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [53]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[39, 20],
        [ 7,  0]],

       [[53,  4],
        [ 8,  1]],

       [[56,  6],
        [ 4,  0]],

       [[48, 15],
        [ 3,  0]],

       [[56,  2],
        [ 8,  0]],

       [[61,  0],
        [ 5,  0]],

       [[59,  1],
        [ 6,  0]],

       [[45, 13],
        [ 8,  0]],

       [[57,  3],
        [ 6,  0]],

       [[60,  1],
        [ 5,  0]],

       [[61,  0],
        [ 5,  0]]], dtype=int64)

In [54]:
accuracy_score(ytrue, yhat)

0.015151515151515152

## 10. Test in Real Time

In [55]:
colors = [(245,117,16), (117,245,16), (16,117,245),(16,117,245),(16,117,245),(16,117,245),(16,117,245),(16,117,245),(16,117,245),(16,117,245),(16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [ ]:
# 1. New detection variables
sequence = []
sentence = []
predictions = []
threshold = 0.9

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        #print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            #print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()